<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `http://....`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vane\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [3]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [4]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [5]:
import nltk
nltk.download('punkt')

docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vane\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [6]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')
print(stop_words)

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vane\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [7]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [8]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [9]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,...,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,...,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [10]:
from sklearn.pipeline import FeatureUnion
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
preprocessor = ColumnTransformer(
    [
        ( "bog",
          CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2), max_features=10000),
          "history_text"),
        ( "Scaler",
            MinMaxScaler(), #O con Standard Scaler
            atributos_de_interes)
        
    ]
)



pipe2 = Pipeline(
    [
    ('wc_tfidfs', preprocessor)
    ]
)

pd.DataFrame(pipe2.fit_transform(df_comics).toarray())

,0,1,2,3,4,5,6,7,8,9,...,9996,9997,9998,9999,10000,10001,10002,10003,10004,10005
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.85,0.3,0.60,0.60,0.40,0.70
1,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.80,1.0,0.80,1.00,1.00,0.80
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.80,0.5,0.55,0.45,1.00,0.55
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.80,0.1,0.25,0.40,0.30,0.50
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.90,0.4,0.45,0.55,0.55,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.90,0.1,0.25,0.30,1.00,0.55
1281,1.0,1.0,0.0,0.0,2.0,0.0,8.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.80,1.0,1.00,1.00,1.00,0.80
1282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.95,0.5,1.00,0.75,1.00,0.80
1283,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.75,0.1,1.00,0.30,1.00,0.30


## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [11]:
from sklearn.dummy import DummyClassifier

MB_pipe = Pipeline(
    [
        ("preprocessing", preprocessor), #aplica bow y minmaxscaler y los concatena
        ("selection", SelectPercentile(f_classif, percentile=90)),
        ("clf", MultinomialNB()),
    ]
)

dummy_pipe = Pipeline(
    [
        ("preprocessing", preprocessor), #aplica bow y minmaxscaler y los concatena
        ("selection", SelectPercentile(f_classif, percentile=90)),
        ("clf", DummyClassifier(strategy="stratified")),
    ]
)


features = df_comics.drop(columns=["alignment"])
labels = df_comics.loc[:, "alignment"]

display(features)
display(labels)

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, shuffle=True, stratify=labels, random_state=42
)

MB_pipe.fit(X_train, y_train)
y_pred = MB_pipe.predict(X_test)
print(classification_report(y_pred, y_test))

dummy_pipe.fit(X_train, y_train)
y_pred = dummy_pipe.predict(X_test)
print(classification_report(y_pred, y_test))

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1362,1445,Zatanna,Zatanna Zatara,Zatanna Zatara,10,Zatanna is the daughter of adventurer John Zat...,Zatanna is genetically talented with her magi...,90,10,25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1363,1446,Zero,DWN-∞: Zero,DWN-∞: Zero,18,Zero was created by the late Dr. Albert Wily ...,NaN,80,100,100,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1364,1447,Zoom (New 52),Hunter Zolomon,NaN,20,"Hunter Zolomon is better known as Zoom, a spee...",After tricking Barry Allen and Wally West into...,95,50,100,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1365,1448,Zoom,Hunter Zolomon,Hunter Zolomon,9,Hunter Zolomon had a troubled relationship wi...,"Zoom is able to alter time, to make himself ev...",75,10,100,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


0       Good
1       Good
2       Good
3       Good
4       Good
        ... 
1362    Good
1363    Good
1364     Bad
1365     Bad
1366     Bad
Name: alignment, Length: 1285, dtype: object

              precision    recall  f1-score   support

         Bad       0.61      0.50      0.55       175
        Good       0.70      0.74      0.72       233
     Neutral       0.11      0.24      0.15        17

    accuracy                           0.62       425
   macro avg       0.47      0.49      0.47       425
weighted avg       0.64      0.62      0.63       425

              precision    recall  f1-score   support

         Bad       0.34      0.36      0.35       132
        Good       0.61      0.57      0.59       263
     Neutral       0.08      0.10      0.09        30

    accuracy                           0.48       425
   macro avg       0.34      0.35      0.34       425
weighted avg       0.49      0.48      0.48       425



En general se ve que la precision es mejor en clasificador no dummy, sin embargo igual son malas. No son suficientes para decir que el clasificador predice bien. Además podemos ver, con el recall que muchas de las variables predichas en verdad son mal predichas, y no corresponden a la categoría donde se clasificó. Los personajes buenos son los que más precisión tienen al clasificarse, pero en cuanto los neutrales y villanos el recalles muy malo. Al considerar que los datos están desbalanceados (con el f1-score) no mejoran mucho las métricas, por lo que no es este el problema.

En conclusión, sí hay una diferencia en el clasificador Multinomial con el Dummy, sin embargo el clasificador Multinomial así como fue presentado no es suficiente y no es buen clasificador, por lo que se debe seguir optimizando.

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [12]:
MB_pipe.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('bog',
                                    CountVectorizer(max_features=10000,
                                                    ngram_range=(1, 2),
                                                    tokenizer=<__main__.StemmerTokenizer object at 0x0000015BD1199630>),
                                    'history_text'),
                                   ('Scaler', MinMaxScaler(),
                                    ['intelligence_score', 'strength_score',
                                     'speed_score', 'durability_score',
                                     'power_score', 'combat_score'])])),
  ('selection', SelectPercentile(percentile=90)),
  ('clf', MultinomialNB())],
 'verbose': False,
 'preprocessing': ColumnTransformer(transformers=[('bog',
                                  CountVectorizer(max_features=10000,
                                                  ngram_range=(1, 2),
             

In [13]:
#### Código aquí ####
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV


params = [
       # clasificador 1 + hiperparámetros
       {'clf': [SVC(random_state=42)],
        'clf__kernel': ["linear"],
        'clf__degree': [1,5,10],
        'preprocessing__bog__ngram_range': [(1,1) , (1,2) , (1,3)], 
        'selection__percentile': [20, 40, 60, 80]
        },
       # clasificador 1 + hiperparámetros    
       {'clf': [KNeighborsClassifier()],
        'clf__n_neighbors': [3, 50, 100],
        'preprocessing__bog__ngram_range': [(1,1) , (1,2) , (1,3)],

        'selection__percentile': [20, 40, 60, 80]
        },
       # clasificador 1 + hiperparámetros
       {'clf': [RandomForestClassifier(random_state=42)],
        'clf__n_estimators': [3, 50, 100],
        'clf__criterion': ['gini', 'entropy'],
        'preprocessing__bog__ngram_range': [(1,1) , (1,2) , (1,3)],
        'selection__percentile': [20, 40, 60, 80]
       }
]


In [14]:
from sklearn.metrics import confusion_matrix

hgs = HalvingGridSearchCV(MB_pipe, params, n_jobs=-1, scoring='f1_weighted')
hgs.fit(X_train, y_train)
y_pred = hgs.predict(X_test)
print("Matriz de confusión: \n")
print(confusion_matrix(y_test, y_pred, labels=hgs.classes_))
print("f1-score", f1_score(y_test, y_pred, average="macro"))
print(classification_report(y_test, y_pred, target_names=hgs.classes_))

C:\Users\Vane\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [0.33238095 0.21666667 0.30357143 0.3968254  0.30833333 0.2452381
 0.27301587 0.29801587 0.30833333 0.26666667 0.1952381  0.27420635
 0.33238095 0.21666667 0.30357143 0.3968254  0.30833333 0.2452381
 0.27301587 0.29801587 0.30833333 0.26666667 0.1952381  0.27420635
 0.33238095 0.21666667 0.30357143 0.3968254  0.30833333 0.2452381
 0.27301587 0.29801587 0.30833333 0.26666667 0.1952381  0.27420635
 0.38238095 0.28333333 0.27142857 0.22857143 0.3        0.28333333
 0.25       0.25       0.3        0.28333333 0.25       0.22142857
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.28253968 0.31111111 0.46

Matriz de confusión: 

[[ 72  60  10]
 [ 47 188  11]
 [ 16  13   8]]
f1-score 0.5012990650313616
              precision    recall  f1-score   support

         Bad       0.53      0.51      0.52       142
        Good       0.72      0.76      0.74       246
     Neutral       0.28      0.22      0.24        37

    accuracy                           0.63       425
   macro avg       0.51      0.50      0.50       425
weighted avg       0.62      0.63      0.62       425



In [16]:
hgs.best_score_

0.6023278167566056

In [17]:


hgs.best_params_



{'clf': SVC(degree=5, kernel='linear', random_state=42),
 'clf__degree': 5,
 'clf__kernel': 'linear',
 'preprocessing__bog__ngram_range': (1, 1),
 'selection__percentile': 80}

El mejor clasificador es el SVC lineal, con grado 5, sin aplicar bi-gramas y con un selection percentile 80. El score entregado por este modelo es de 60%, este si bien no es malo, no es de los mejores, pero es probable que esto se deba al problema que hubo con la codificacion de las palabras del dataset, error que como ya se mencionó no se pudo arreglar. La otra posibilidad es que los clasificadores e hiperparámetros elegidos no hayan sido los correctos, sin embargo se probó con varias opciones. 

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [19]:
df_comics_no_label

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,14,Ace Morgan,Kyle Morgan,NaN,7,NaN,"Aviation: Ace is an extremely skilled pilot, ...",85,20,65,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
2,17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1394,Weapon Hex,Laura Kinney,NaN,-,NaN,NONE NOTABLE,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,1396,Wendigo,Wendigo,NaN,6,The Wendigo are the result of an ancient curse...,NaN,65,80,75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
81,1431,XS (CW),Nora West-Allen,NaN,19,"According to Eobard Thawne, the daughter of Ba...",Nora possesses a natural connection to the Spe...,90,55,100,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
82,1440,Yondu (MCU),Yondu,NaN,7,"Yondu Udonta was a Centaurian, leader of a key...","Yondu Udonta was a member of the Centaurians, ...",85,25,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:

final_preprocessor = ColumnTransformer(
    [
        ( "bog",
          CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,1), max_features=10000),
          "history_text"),
        ( "Scaler",
            MinMaxScaler(), #O con Standard Scaler
            atributos_de_interes)
        
    ]
)

final_pipe = Pipeline(
    [
        ("preprocessing", final_preprocessor), #aplica bow y minmaxscaler y los concatena
        ("selection", SelectPercentile(f_classif, percentile=80)),
        ("clf", SVC(degree=5, kernel='linear', random_state=42)),
    ]
)
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text'])
features = df_comics.drop(columns=["alignment"])
labels = df_comics.loc[:, "alignment"]

final_pipe.fit(features, labels) #En este caso se utilizan todos los datos con labels para entrenar
y_pred = final_pipe.predict(df_comics_no_label)

display(y_pred)

array(['Good', 'Bad', 'Good', 'Good', 'Bad', 'Good', 'Neutral', 'Good',
       'Bad', 'Bad', 'Good', 'Good', 'Bad', 'Bad', 'Good', 'Good',
       'Neutral', 'Neutral', 'Bad', 'Good', 'Good', 'Good', 'Good', 'Bad',
       'Good', 'Good', 'Good', 'Bad', 'Good', 'Bad', 'Bad', 'Good',
       'Good', 'Good', 'Bad', 'Good', 'Bad', 'Bad', 'Good', 'Good', 'Bad',
       'Good', 'Good', 'Good', 'Bad', 'Bad', 'Bad', 'Neutral', 'Bad',
       'Good', 'Good', 'Good', 'Bad', 'Bad', 'Good', 'Bad', 'Neutral',
       'Neutral', 'Good', 'Good', 'Good', 'Bad', 'Bad', 'Bad', 'Neutral',
       'Good', 'Bad', 'Good', 'Good', 'Bad', 'Good', 'Good', 'Neutral',
       'Good', 'Good', 'Good'], dtype=object)

In [24]:
df = pd.DataFrame()
df["name"] = df_comics_no_label["name"]
df["pred"] = y_pred
df

,name,pred
0,514A (Gotham),Good
2,A'dal,Bad
3,Agent Zero (FOX),Good
4,Ajax (FOX),Good
6,A.M.A.Z.O. (CW),Bad
...,...,...
78,Vergil,Good
80,Wendigo,Neutral
81,XS (CW),Good
82,Yondu (MCU),Good


<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

BatCow finalmente era héroe :)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>